In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import scipy.stats as stats
import requests

In [18]:
#genius api
genius_api_key = 'gQVkGL-wnevLS3t39UOjl1a9x3smebFPcMX6awdjS-61kw-NrQEQmWWDbNXzUWjz'
genius_url = 'https://api.genius.com'
headers = {'authorization': f'Bearer {genius_api_key}'}

In [ ]:
spotify_songs_csv = Path("Resources/spotify_top_songs.csv")
spotify_songs_df = pd.read_csv(spotify_songs_csv)
spotify_songs_df.drop (columns=["Top 10 (xTimes)", 'Peak Position', 'Peak Position (xTimes)', 'Peak Streams', ], inplace=True)

spotify_songs_df.head()

,Position,Artist Name,Song Name,Days,Total Streams
0,1,Post Malone,Sunflower SpiderMan: Into the SpiderVerse,1506,883369738
1,2,Juice WRLD,Lucid Dreams,1673,864832399
2,3,Lil Uzi Vert,XO TOUR Llif3,1853,781153024
3,4,J. Cole,No Role Modelz,2547,734857487
4,5,Post Malone,rockstar,1223,718865961


In [ ]:
genius_search = f"genius_url/search"
params = {'q':"Sunflower  SpiderMan: Into the SpiderVerse"}
genius_response = requests.get(genius_search,headers=headers, params=params)
for hit in data['genius_response']['hits']:
    